In [1]:
import pandas as pd

# Cargar el archivo CSV en un nuevo DataFrame
df_combined = pd.read_csv('df_combined.csv')

# Mostrar las primeras filas para verificar que se ha cargado correctamente
print(df_combined.head())

         date  price_bitcoin  price_gold  price_sp500  change_bitcoin  \
0  2015-01-02            315        1186         2058            0.10   
1  2015-01-05            275        1204         2021            3.82   
2  2015-01-06            282        1219         2003            2.70   
3  2015-01-07            291        1211         2026            3.21   
4  2015-01-08            283        1208         2062           -2.97   

   change_gold    vix  interest_rate     cpi  inflation  
0         0.18  17.79           0.01  234.82       0.76  
1         1.50  19.92           0.00  234.84       0.76  
2         1.28  21.12           0.02  234.84       0.76  
3        -0.71  19.31           0.02  234.84       0.76  
4        -0.18  17.01           0.02  234.85       0.76  
